In [6]:
import numpy as np
"""
Q:
Write a binomial tree program to calculate the put prices of Bermuda options. 
For such options, early exercise is allowed only on specific dates.

Inputs: 
S (stock price)
X (strike price)
r (continuously compounded annual interest rate in percentage)
s (annual volatility in percentage)
T (time to maturity in days, which of course is also an exercise date)
E (set of early exercise dates from now)
m (number of periods per day for the tree)

Output:
11.2486

Long put:MAX{X-ST,0} 
X (Strike Price)
ST (represent the price of the underlying at maturity)
"""

S = 100
X = 95
r = 3 #3%
s = 15 #30%
T = 365
E = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360]
m = 2

class BermudaOptions_BinomialTree:
    def __init__(self,S,X,r,s,T,E,m):
        self.S = S
        self.X = X
        self.r = r/100
        self.s = s/100
        self.T = T
        self.E = E
        if T not in self.E:
            self.E.append(T)
        self.m = m
        self.deltaT = (self.T/365)/(self.T*self.m)
        self.R = np.exp(self.r*self.deltaT)
        
        self.u = np.exp(self.s*np.sqrt(self.deltaT))
        self.d = 1/self.u
        self.p = (self.R - self.d) / (self.u-self.d) #Risk Neutral Probability 
        
    class node:
        def __init__(self,price):
            self.price = price
            self.value = 0
            self.u_parent = None
            self.d_parent = None
            self.u_child = None
            self.d_child = None
            
    def price(self):
        self.root = self.node(self.S)
        self.levels = [[self.root]]
        total = self.T*self.m
        
        for period in range(self.T*self.m):
            currentLevel = self.levels[-1]
            childLevel = [BermudaOptions_BinomialTree.node(price=currentLevel[0].price * self.u)]
            for currentNode in currentLevel:
                currentNode.u_child = childLevel[-1]
                currentNode.u_child.d_parent = currentNode
                downChild = BermudaOptions_BinomialTree.node(price=currentNode.price * self.d)
                downChild.u_parent = currentNode
                currentNode.d_child = downChild
                childLevel.append(downChild)
            self.levels.append(childLevel)
        for levelNodes in self.levels[::-1]:
            if int(total/self.m) in E:
                for node in levelNodes:
                    if node.u_child != None:
                        binomialValue = ( ( 1/self.R ) * (self.p * node.u_child.value + (1 - self.p) * node.d_child.value))
                    else:

                        exerciseValue = max(0,self.X - node.price)
                        binomialValue = exerciseValue
                        #print(exerciseValue)

                    exerciseValue = max(0,self.X - node.price)
                    node.value = max(binomialValue, exerciseValue)
            else:
                for node in levelNodes:
                    binomialValue = ( ( 1/self.R ) * (self.p * node.u_child.value + (1 - self.p) * node.d_child.value))
                    node.value = binomialValue
            total -= 1
                
        putPrice = self.root.value
        
        return putPrice,self.levels

price,tree = BermudaOptions_BinomialTree(S,X,r,s,T,E,m).price()
print("price=",price)

price= 2.802827724666218


In [10]:
import numpy as np

"""
Q:
Write a binomial tree program to calculate the put prices of Bermuda options. 
For such options, early exercise is allowed only on specific dates.

Inputs: 
S (stock price)
X (strike price)
r (continuously compounded annual interest rate in percentage)
s (annual volatility in percentage)
T (time to maturity in days, which of course is also an exercise date)
E (set of early exercise dates from now)
m (number of periods per day for the tree)

Output:
11.2486

Long put:MAX{X-ST,0} 
X (Strike Price)
ST (represent the price of the underlying at maturity)
"""

class node:
    def __init__(self,price):
        self.price = price
        self.value = 0
        self.u_parent = None
        self.d_parent = None
        self.u_child = None
        self.d_child = None

def price(S,X,r,s,T,E,m):
    if T not in E:
        E.append(T)
    deltaT = (T/365)/(T*m)
    R = np.exp(r*deltaT)
    u = np.exp(s*np.sqrt(deltaT))
    d = 1/u
    p = (R - d) / (u-d) #Risk Neutral Probability 
    root = node(S)
    BinomialTree = [[root]]
    
    for period in range(T*m):
        currentLevel = BinomialTree[-1]
        childLevel = [node(currentLevel[0].price * u)]
        #print(childLevel[0].price)
        for currentNode in currentLevel:
            currentNode.u_child = childLevel[-1]
            currentNode.u_child.d_parent = currentNode
            downChild = node(currentNode.price * d)
            downChild.u_parent = currentNode
            currentNode.d_child = downChild
            childLevel.append(downChild)
        BinomialTree.append(childLevel)
        
    total = T * m
    for levelNodes in BinomialTree[::-1]:
        if int(total/m) in E:
            for tree_node in levelNodes:
                if tree_node.u_child != None:
                    binomialValue = ( ( 1/R ) * (p * tree_node.u_child.value + (1 - p) * tree_node.d_child.value))
                else:
                    exerciseValue = max(0,X - tree_node.price)
                    binomialValue = exerciseValue
                    #print(exerciseValue)

                exerciseValue = max(0,X - tree_node.price)
                tree_node.value = max(binomialValue, exerciseValue)
        else:
            for tree_node in levelNodes:
                binomialValue = ( ( 1/R ) * (p * tree_node.u_child.value + (1 - p) * tree_node.d_child.value))
                tree_node.value = binomialValue
        total -= 1

    putPrice = root.value

    return putPrice

if __name__ == "__main__":
    S = float(input("輸入Stock Price ex:100\n"))
    X = float(input("輸入Strike price ex:110\n"))
    r = float(input("輸入Continuously compounded annual interest rate in percentage(%) ex:3\n"))/100
    s = float(input("輸入Annual volatility in percentage(%) ex:30\n"))/100
    T = int(input("輸入time to maturity in days,which of course is also an exercise date ex:60\n"))
    Temp = input("輸入set of early exercise dates from now ex:10 20 30 40 50\n").split(' ')
    E = []
    for temp in Temp:
        E.append(int(temp))
    m = int(input("輸入number of periods per day for the tree ex:5\n"))
    print("S= %f" %S)
    print("X= %f" %X) 
    print("r= %f" %r)
    print("s= %f" %s)
    print("T= %d" %T)
    print("E=",E)
    print("m= %d" %m)
    
    output = price(S,X,r,s,T,E,m)
    print("The put price of Bermuda option is about =",output)

輸入Stock Price ex:100
100
輸入Strike price ex:110
95
輸入Continuously compounded annual interest rate in percentage(%) ex:3
3
輸入Annual volatility in percentage(%) ex:30
15
輸入time to maturity in days,which of course is also an exercise date ex:60
365
輸入set of early exercise dates from now ex:10 20 30 40 50
10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360
輸入number of periods per day for the tree ex:5
2
S= 100.000000
X= 95.000000
r= 0.030000
s= 0.150000
T= 365
E= [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360]
m= 2
The put price of Bermuda option is about = 2.802827724666218
